# IRIS

Classical SVM - sklearn

QSVM - qiskit (based on ideas from https://github.com/Qiskit/qiskit-machine-learning/blob/main/docs/tutorials/03_quantum_kernel.ipynb)

In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
np.random.seed(42)

## Data Prep

In [3]:
iris = load_iris()

In [4]:
iris['data'].shape

(150, 4)

In [5]:
X = iris.data
Y = iris.target

In [6]:
# set aside 20% of train and test data for evaluation
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle = True, random_state = 42)

### Training data

In [7]:
# Fitting and scaling training data
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)

### Validation data

In [8]:
# Scaling validation data
x_test = scaler.transform(x_test)

## SVM Training

In [9]:
%%time

kernel_list = ['linear', 'poly', 'rbf', 'sigmoid']

for kernel in kernel_list:
    svc_classifier = SVC(kernel = kernel)
    svc_classifier.fit(x_train,y_train)
    
    train_score = svc_classifier.score(x_train,y_train)
    test_score = svc_classifier.score(x_test,y_test)
    
    print(f'Kernel: {kernel}, train score: {train_score}, test score: {test_score}')
    

Kernel: linear, train score: 0.9833333333333333, test score: 0.9666666666666667
Kernel: poly, train score: 0.95, test score: 0.9666666666666667
Kernel: rbf, train score: 0.9666666666666667, test score: 1.0
Kernel: sigmoid, train score: 0.8916666666666667, test score: 0.9
CPU times: user 4.58 ms, sys: 910 µs, total: 5.49 ms
Wall time: 4.33 ms


## QSVC training

In [10]:
from qiskit import version
from qiskit.providers.aer.backends import StatevectorSimulator
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import QuantumKernel

seed = 42
algorithm_globals.random_seed = seed

In [11]:
version.QiskitVersion()

{'qiskit-terra': '0.19.2', 'qiskit-aer': '0.10.3', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.18.3', 'qiskit-aqua': None, 'qiskit': '0.34.2', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': '0.3.1'}

In [12]:
# number of features in the data - corresponds to number of qubits used 
iris_data_features = x_train.shape[1]
# how many times to repeat the quantum embedding circuit
reps = 1
# what type of entanglement between qubits to use with ZZFeatureMap
# ('full', 'linear', 'circular', 'sca')
entanglement="full"

## Using ZFeatureMap for quantum embedding of classical data

In [13]:
iris_feature_map = ZFeatureMap(feature_dimension=iris_data_features, reps=reps)

quantum_simulator = StatevectorSimulator(device='CPU')

quantum_backend = QuantumInstance(
    quantum_simulator, seed_simulator=seed, seed_transpiler=seed
)

iris_kernel = QuantumKernel(feature_map=iris_feature_map, quantum_instance=quantum_backend)

In [14]:
%%time
iris_matrix_train = iris_kernel.evaluate(x_vec=x_train)

CPU times: user 518 ms, sys: 62.4 ms, total: 580 ms
Wall time: 879 ms


In [15]:
iris_matrix_train.shape

(120, 120)

In [16]:
%%time
iris_svc = SVC(kernel="precomputed")
iris_svc.fit(iris_matrix_train, y_train)

CPU times: user 820 µs, sys: 0 ns, total: 820 µs
Wall time: 627 µs


SVC(kernel='precomputed')

In [17]:
%%time
iris_train_score = iris_svc.score(iris_matrix_train, y_train)
print(f"Precomputed kernel classification train score: {iris_train_score}")

Precomputed kernel classification train score: 0.95
CPU times: user 1.35 ms, sys: 0 ns, total: 1.35 ms
Wall time: 775 µs


In [18]:
%%time
iris_matrix_test = iris_kernel.evaluate(x_vec=x_test, y_vec=x_train)
iris_test_score = iris_svc.score(iris_matrix_test, y_test)
print(f"Precomputed kernel classification test score: {iris_test_score}")

Precomputed kernel classification test score: 0.9666666666666667
CPU times: user 191 ms, sys: 41.1 ms, total: 232 ms
Wall time: 228 ms


## Using ZZFeatureMap for quantum embedding of classical data

In [19]:
iris_feature_map = ZZFeatureMap(feature_dimension=iris_data_features, reps=reps, entanglement=entanglement)

quantum_simulator = StatevectorSimulator(device='CPU')

quantum_backend = QuantumInstance(
    quantum_simulator, seed_simulator=seed, seed_transpiler=seed
)

iris_kernel = QuantumKernel(feature_map=iris_feature_map, quantum_instance=quantum_backend)

In [20]:
%%time
iris_matrix_train = iris_kernel.evaluate(x_vec=x_train)

CPU times: user 464 ms, sys: 33 ms, total: 497 ms
Wall time: 494 ms


In [21]:
%%time
iris_svc = SVC(kernel="precomputed")
iris_svc.fit(iris_matrix_train, y_train)

CPU times: user 1.36 ms, sys: 938 µs, total: 2.29 ms
Wall time: 1.11 ms


SVC(kernel='precomputed')

In [22]:
%%time
iris_train_score = iris_svc.score(iris_matrix_train, y_train)
print(f"Precomputed kernel classification train score: {iris_train_score}")

Precomputed kernel classification train score: 0.9583333333333334
CPU times: user 310 µs, sys: 258 µs, total: 568 µs
Wall time: 390 µs


In [23]:
%%time
iris_matrix_test = iris_kernel.evaluate(x_vec=x_test, y_vec=x_train)
iris_test_score = iris_svc.score(iris_matrix_test, y_test)
print(f"Precomputed kernel classification test score: {iris_test_score}")

Precomputed kernel classification test score: 0.6666666666666666
CPU times: user 379 ms, sys: 43.7 ms, total: 423 ms
Wall time: 419 ms


## References

- https://github.com/Qiskit/qiskit-machine-learning/blob/main/docs/tutorials/03_quantum_kernel.ipynb
- https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZFeatureMap.html
- https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZZFeatureMap.html
- https://qiskit.org/documentation/machine-learning/stubs/qiskit_machine_learning.kernels.QuantumKernel.html
- https://qiskit.org/documentation/stubs/qiskit.providers.aer.StatevectorSimulator.html
- https://qiskit.org/documentation/stubs/qiskit.utils.QuantumInstance.html?highlight=quantuminstance#qiskit.utils.QuantumInstance
